In [ ]:
%matplotlib inline
import cv2
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
from random import shuffle
from tensorflow.keras.applications import VGG16
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation
import sys
import h5py

In [ ]:
def print_percentage(curr, final):
    # Percentage completion.
    p = curr / final
    msg = "\r- Percentage: {0:.1%}".format(p)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [ ]:
# Frame size  
image_size = 224

image_size_touple = (image_size, image_size)

# Number of channels (RGB)
n_channels = 3

# Number of classes for classification (Violence-No Violence)
num_classes = 2

# Number of frames per video
frames_per_file = 20

# Video extension
video_extension = ".avi"

data_path = "hockey-fight-videos/data"

In [ ]:
def load_rgb_frames(current_dir, file_name):
    
    in_file = os.path.join(current_dir, file_name)
    
    images = []
    
    vidcap = cv2.VideoCapture(in_file)
    
    success,image = vidcap.read()
        
    count = 0

    while count<frames_per_file:
                
        RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        res = cv2.resize(RGB_img, dsize=(image_size, image_size),
                                 interpolation=cv2.INTER_CUBIC)
    
        images.append(res)
    
        success,image = vidcap.read()
    
        count += 1
        
    resul = np.array(images)
    
    resul = (resul / 255.).astype(np.float16)
        
    return resul

In [ ]:
def get_labels(data_path):
    
    names = []

    labels = []
    
    
    for current_dir, dir_names, file_names in os.walk(data_path):
        
        for file_name in file_names:
            
            if file_name[0:2] == 'fi':
                labels.append([1,0])
                names.append(file_name)
            elif file_name[0:2] == 'no':
                labels.append([0,1])
                names.append(file_name)
                     
            
    c = list(zip(names,labels))

    shuffle(c)
    
    names, labels = zip(*c)
            
    return names, labels

In [ ]:

names, labels = get_labels(data_path)

In [ ]:
image_model = VGG16(include_top=True, weights='imagenet')

In [ ]:
# We will use the output of the layer prior to the final
# classification-layer which is named fc2. This is a fully-connected (or dense) layer.
transfer_layer = image_model.get_layer('block5_conv3')

image_model_transfer = Model(inputs=image_model.input,
                             outputs=transfer_layer.output)

transfer_values_size = K.int_shape(image_model_transfer.output)


print("The input of the VGG16 net have dimensions:",K.int_shape(image_model.input))

print("The output of the selected layer of VGG16 net have dimensions: ", transfer_values_size)

In [ ]:
import os
os.mkdir('/kaggle/working/train')
os.mkdir('/kaggle/working/valid')
os.mkdir('/kaggle/working/test')

In [ ]:
def load_data_train(vid_names, data_path, labels):
    
    count = 0
    
    tam = len(vid_names)
    
    shape = (frames_per_file,) + image_size_touple + (1,)
    
    while count<tam:
        
        video_name = vid_names[count]
        
        image_batch = np.zeros(shape=shape, dtype=np.float16)
    
        image_batch = load_rgb_frames(data_path, video_name)
        
        shape = (frames_per_file, 14, 14, 512)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values = \
            image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([20,2])
        
        labelss = labels1*aux
        
        file_path = 'train/data' + str(count+1) + '.h5'
        
        with h5py.File(file_path, 'w') as hf:
            hf.create_dataset('data', data = transfer_values)
            hf.create_dataset('labels', data = labels[count])
        
        count+=1

In [ ]:
def load_data_valid(vid_names, data_path, labels):
    
    count = 0
    
    tam = len(vid_names)
    
    shape = (frames_per_file,) + image_size_touple + (1,)
    
    while count<tam:
        
        video_name = vid_names[count]
        
        image_batch = np.zeros(shape=shape, dtype=np.float16)
    
        image_batch = load_rgb_frames(data_path, video_name)
        
        shape = (frames_per_file, 14, 14, 512)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values = \
            image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([20,2])
        
        labelss = labels1*aux
        
        file_path = 'valid/data' + str(count+1) + '.h5'
        
        with h5py.File(file_path, 'w') as hf:
            hf.create_dataset('data', data = transfer_values)
            hf.create_dataset('labels', data = labels[count])
        
        count+=1

In [ ]:
def load_data_test(vid_names, data_path, labels):
    
    count = 0
    
    tam = len(vid_names)
    
    shape = (frames_per_file,) + image_size_touple + (1,)
    
    while count<tam:
        
        video_name = vid_names[count]
        
        image_batch = np.zeros(shape=shape, dtype=np.float16)
    
        image_batch = load_rgb_frames(data_path, video_name)
        
        shape = (frames_per_file, 14, 14, 512)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values = \
            image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([20,2])
        
        labelss = labels1*aux
        
        file_path = 'test/data' + str(count+1) + '.h5'
        
        with h5py.File(file_path, 'w') as hf:
            hf.create_dataset('data', data = transfer_values)
            hf.create_dataset('labels', data = labels[count])
        
        count+=1

In [ ]:
load_data_train(names[0:720], data_path, labels[0:720])
load_data_valid(names[720:800], data_path, labels[720:800])
load_data_test(names[800:1000], data_path, labels[800:1000])

In [ ]:
os.mkdir('/kaggle/working/train1')
os.mkdir('/kaggle/working/valid1')
os.mkdir('/kaggle/working/test1')

In [ ]:
def threshold_based_flow_accumulation(current_dir, file_name):
    
    in_file = os.path.join(current_dir, file_name)
    
    images = []
    
    vidcap = cv2.VideoCapture(in_file)
    
    success,image = vidcap.read()
        
    count = 0
    
    flow_result = []
    
    flow_frame = np.zeros((224, 224))
    
    mod_frame = np.zeros((224, 224))
    
    mod_frame.fill(1.25)

    while count<21:
        
        RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        res = cv2.resize(RGB_img, dsize=(image_size, image_size),
                                     interpolation=cv2.INTER_CUBIC)
        images.append(res)
        
        success,image = vidcap.read()
        
        if(count>0):
                
            diff_frame = cv2.absdiff(images[count], images[count-1])
            res1, threshold_frame1 = cv2.threshold(diff_frame,25,255,cv2.THRESH_BINARY)
            flow_frame = flow_frame + threshold_frame1
            res2, threshold_frame2 = cv2.threshold(flow_frame, 255, 255, cv2.THRESH_TRUNC)
            stacked_img = np.stack((threshold_frame2,)*3, axis=-1)
            flow_result.append(stacked_img)
            flow_frame = flow_frame/mod_frame
        
    
        count += 1
    
    answer = np.array(flow_result)
    
    answer = (answer / 255.).astype(np.float16)
    
    return answer

In [ ]:
def load_data_train_flow(vid_names, data_path, labels):
    
    count = 0
    
    tam = len(vid_names)
    
    shape = (frames_per_file,) + image_size_touple + (1,)
    
    while count<tam:
        
        video_name = vid_names[count]
        
        image_batch = np.zeros(shape=shape, dtype=np.float16)
    
        image_batch = threshold_based_flow_accumulation(data_path, video_name)
        
        shape = (frames_per_file, 14, 14, 512)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values = \
            image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([20,2])
        
        labelss = labels1*aux
        
        file_path = 'train1/data' + str(count+1) + '.h5'
        
        with h5py.File(file_path, 'w') as hf:
            hf.create_dataset('data', data = transfer_values)
            hf.create_dataset('labels', data = labels[count])
        
        count+=1

In [ ]:
def load_data_valid_flow(vid_names, data_path, labels):
    
    count = 0
    
    tam = len(vid_names)
    
    shape = (frames_per_file,) + image_size_touple + (1,)
    
    while count<tam:
        
        video_name = vid_names[count]
        
        image_batch = np.zeros(shape=shape, dtype=np.float16)
    
        image_batch = threshold_based_flow_accumulation(data_path, video_name)
        
        shape = (frames_per_file, 14, 14, 512)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values = \
            image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([20,2])
        
        labelss = labels1*aux
        
        file_path = 'valid1/data' + str(count+1) + '.h5'
        
        with h5py.File(file_path, 'w') as hf:
            hf.create_dataset('data', data = transfer_values)
            hf.create_dataset('labels', data = labels[count])
        
        count+=1

In [ ]:
def load_data_test_flow(vid_names, data_path, labels):
    
    count = 0
    
    tam = len(vid_names)

    shape = (frames_per_file,) + image_size_touple + (1,)
    
    while count<tam:
        
        video_name = vid_names[count]
        
        image_batch = np.zeros(shape=shape, dtype=np.float16)
    
        image_batch = threshold_based_flow_accumulation(data_path, video_name)
        
        shape = (frames_per_file, 14, 14, 512)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values = \
            image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([20,2])
        
        labelss = labels1*aux
        
        file_path = 'test1/data' + str(count+1) + '.h5'
        
        with h5py.File(file_path, 'w') as hf:
            hf.create_dataset('data', data = transfer_values)
            hf.create_dataset('labels', data = labels[count])
        
        count+=1

In [ ]:
load_data_train_flow(names[0:720], data_path, labels[0:720])
load_data_valid_flow(names[720:800], data_path, labels[720:800])
load_data_test_flow(names[800:1000], data_path, labels[800:1000])